In [ ]:
#| default_exp tools

# LLM tools
> Helpful tools for running cli commands and reading, modifying, and creating files in python. This is used primarily for AI's in tool loops for automating tasks involving the filesystem.

In [ ]:
#| export
from fastcore.xdg import *
from fastcore.imports import *
from fastcore.xtras import truncstr
from shlex import split
from subprocess import run, DEVNULL

In [ ]:
#| hide
from fastcore.test import test_eq,test_fail
from toolslm.funccall import get_schema
import inspect

## Error handling helpers


In [ ]:
#| export
def explain_exc(task=''):
    """Convert an current exception to  an LLM friendly error message."""
    try: raise sys.exc_info()[1]
    except (AssertionError, ZeroDivisionError, ValueError, FileNotFoundError) as e:
        return f"Error: {e}"
    except Exception as e: return f"Error {task}: {repr(e)}"


In [ ]:
def calc_div(a:int,b:int):
    "Divides a/b - example tool"
    try: return a/b
    except: return explain_exc("dividing")
calc_div(1,0)

'Error: division by zero'

In [ ]:
#| export
def ensure(b: bool, msg:str=""):
    "Works like assert b, msg but raise ValueError and is not disabled when run with python -O"
    if not b: raise ValueError(msg)


In [ ]:
def calc_div(a:int,b:int):
    "Divides a/b - example tool"
    try:
        ensure(b != 0, "B cannot be zero") 
        return a/b
    except: return explain_exc("dividing")
calc_div(1,0)

'Error: B cannot be zero'

In [ ]:
#| export
def _load_valid_paths():
    cfg = xdg_config_home() / 'fc_tools_paths'
    base = ['.', '/tmp']
    if not cfg.exists(): return base
    return base + cfg.read_text().split()

valid_paths = _load_valid_paths()

In [ ]:
#| export
def valid_path(path:str, must_exist:bool=True, chk_perms:bool=True) -> Path:
    'Return expanded/resolved Path, raising FileNotFoundError if must_exist and missing'
    p = Path(path).expanduser().resolve()
    vpaths = [Path(vp).expanduser().resolve() for vp in valid_paths]
    if chk_perms and not any(p == vp or vp in p.parents for vp in vpaths): raise PermissionError(f'Path not in valid_paths: {p}')
    if must_exist and not p.exists(): raise FileNotFoundError(f'File not found: {p}')
    return p

In [ ]:
assert valid_path('.')
assert valid_path('/tmp')

test_fail(lambda: valid_path('..'), exc=PermissionError)

In [ ]:
try: valid_path(".missing")
except: print(explain_exc())

Error: File not found: /Users/jhoward/aai-ws/fastcore/nbs/.missing


## Bash Tools

In [ ]:
#| export
def run_cmd(
    cmd:str, # The command name to run
    argstr:str='', # All args to the command, will be split with shlex
    disallow_re:str=None, # optional regex which, if matched on argstr, will disallow the command
    allow_re:str=None # optional regex which, if not matched on argstr, will disallow the command
):
    "Run `cmd` passing split `argstr`, optionally checking for allowed argstr"
    try:
        ensure(not (disallow_re and re.search(disallow_re, argstr)), 'args disallowed')
        ensure(not (allow_re    and re.search(   allow_re, argstr)), 'args not allowed')
        argstr = argstr.replace('\\', '\\\\')
        args = [str(Path(a).expanduser()) if a.startswith('~') else a for a in split(argstr, posix=True)]
        outp = run([cmd] + args, text=True, stdin=DEVNULL, capture_output=True)
    except: return explain_exc(f'running cmd')
    res = outp.stdout
    if res and outp.stderr: res += '\n'
    return res + outp.stderr

With this little function, we can now run any cli command:

In [ ]:
print(run_cmd('ls', '~')[:128])

aai-ws
Applications
autoexec.ipynb
books
cachy.jsonl
chats
CRAFT.ipynb
Desktop


Note that, for tool safety, this is not passed through the shell, so wildcards, env vars, etc will not work (although `~` will):

In [ ]:
print(run_cmd('ls', 'f*')[:128])

ls: f*: No such file or directory



Let's create some useful functions from this that will allow for searching, reading and modifing content on the file system.

In [ ]:
#| export
@llmtool
def rg(
    argstr:str, # All args to the command, will be split with shlex
    disallow_re:str=None, # optional regex which, if matched on argstr, will disallow the command
    allow_re:str=None # optional regex which, if not matched on argstr, will disallow the command
):
    "Run the `rg` command with the args in `argstr`"
    return run_cmd('rg', '-n '+argstr, disallow_re=disallow_re, allow_re=allow_re)

In [ ]:
rg('fast.ai CNAME')

'1:fastcore.fast.ai\n'

In [ ]:
print(rg('"^\s+.def ' + r'rg\(" 12_tools.ipynb'))

347:    "def rg(\n",



Functions implemented with `run_cmd` like this one can be passed regexps to allow or disallow arg strs, i.e to block parent or root directories:

In [ ]:
disallowed = r' /|\.\.'
rg('info@fast.ai ..', disallow_re=disallowed)

'Error: args disallowed'

In [ ]:
rg('info@fast.ai /', disallow_re=disallowed)

'Error: args disallowed'

In [ ]:
print(rg('fast.ai CNAME', disallow_re=disallowed))

1:fastcore.fast.ai



NB: These tools have special behavior around errors. Since these have been speficially designed for work with LLMs, any exceptions created from their use is returned as a string to help them debug their work.

In [ ]:
run_cmd('asdfe')

"Error: [Errno 2] No such file or directory: 'asdfe'"

In [ ]:
#| export
@llmtool
def sed(
    argstr:str, # All args to the command, will be split with shlex
    disallow_re:str=None, # optional regex which, if matched on argstr, will disallow the command
    allow_re:str=None # optional regex which, if not matched on argstr, will disallow the command
):
    "Run the `sed` command with the args in `argstr` (e.g for reading a section of a file)"
    return run_cmd('sed', argstr, allow_re=allow_re, disallow_re=disallow_re)

In [ ]:
print(sed('-n "1,5 p" _quarto.yml'))

project:
  type: website
  pre-render: 
    - pysym2md --output_file apilist.txt fastcore
  post-render: 



In [ ]:
# Print line numbers too
print(sed('-n "1,5 {=;p;}" _quarto.yml'))

1
project:
2
  type: website
3
  pre-render: 
4
    - pysym2md --output_file apilist.txt fastcore
5
  post-render: 



## Text Edit Tools

Python implementations of the text editor tools from [Anthropic](https://docs.claude.com/en/docs/agents-and-tools/tool-use/text-editor-tool), plus more. These tools are especially useful in an AI's tool loop. See [`claudette`](https://claudette.answer.ai/text_editor.html) for examples.

In [ ]:
#| export
def _fmt_path(f, p, skip_folders=()):
    'Format path with emoji for dirs/symlinks or size for files'
    parts = f.relative_to(p).parts
    if any(part.startswith('.') for part in parts): return None
    if any(part in skip_folders for part in parts): return None
    if f.is_symlink(): return f'{f} 🔗'
    if f.is_dir(): return f'{f} 📁'
    return f'{f} ({f.stat().st_size/1024:.1f}k)'

In [ ]:
#| export
@llmtool
def view(
    path:str, # Path to directory or file to view
    view_range:tuple[int,int]=None, # Optional 1-indexed (start, end) line range for files, end=-1 for EOF. Do NOT use unless it's known that the file is too big to keep in context—simply view the WHOLE file when possible
    nums:bool=False, # Whether to show line numbers
    skip_folders:tuple[str,...]=('_proc','__pycache__') # Folder names to skip when listing directories
):
    'View directory or file contents with optional line range and numbers'
    try:
        p = valid_path(path, chk_perms=False)
        header = None
        if p.is_dir():
            lines = [s for f in p.glob('**/*') if (s := _fmt_path(f, p, skip_folders))]
            header = f'Directory contents of {p}:'
        else: lines = p.read_text().splitlines()
        s, e = 1, len(lines)
        if view_range:
            s,e = view_range
            ensure(1<=s<=len(lines), f'Invalid start line {s}')
            ensure(e==-1 or s<=e<=len(lines), f'Invalid end line {e}')
            lines = lines[s-1:None if e==-1 else e]
        if nums: lines = [f'{i+s:6d} │ {l}' for i, l in enumerate(lines)]
        content = '\n'.join(lines)
        return f'{header}\n{content}' if header else content
    except: return explain_exc('viewing')

You can specify line ranges and whether to have the output contain line numbers:

In [ ]:
print(view('_quarto.yml', (1,10), nums=True))

     1 │ project:
     2 │   type: website
     3 │   pre-render: 
     4 │     - pysym2md --output_file apilist.txt fastcore
     5 │   post-render: 
     6 │     - llms_txt2ctx llms.txt --optional true --save_nbdev_fname llms-ctx-full.txt
     7 │     - llms_txt2ctx llms.txt --save_nbdev_fname llms-ctx.txt
     8 │   resources: 
     9 │     - "*.txt"
    10 │   preview:


Here's what the output looks like when viewing a directory:

In [ ]:
print(view('.', (1,5)).replace(os.getcwd(), '/path'))

Directory contents of /path:
/path/llms.txt (3.7k)
/path/000_tour.ipynb (18.2k)
/path/parallel_test.py (0.6k)
/path/_quarto.yml (0.8k)
/path/08_style.ipynb (12.3k)


In [ ]:
#| export
@llmtool
def create(
    path: str, # Path where the new file should be created
    file_text: str, # Content to write to the file
    overwrite:bool=False # Whether to overwrite existing files
) -> str:
    'Creates a new file with the given content at the specified path'
    try:
        p = valid_path(path, must_exist=False)
        if p.exists():
            if not overwrite: return f'Error: File already exists: {p}'
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(file_text)
        return f'Created file {p}.'
    except: return explain_exc('creating file')

In [ ]:
print(create('test.txt', 'Hello, world!').replace(os.getcwd(), '/path'))
f = Path('test.txt')
test_eq(f.exists(), True)
print('Contents:\n', view(f, nums=True))

Created file /path/test.txt.
Contents:
      1 │ Hello, world!


In [ ]:
#| export
@llmtool
def insert(
    path: str, # Path to the file to modify
    insert_line: int, # Line number where to insert (0-based indexing)
    new_str: str # Text to insert at the specified line
) -> str:
    'Insert new_str at specified line number'
    try:
        p = valid_path(path)
        content = p.read_text().splitlines()
        ensure(0<=insert_line<=len(content), f'Invalid line number {insert_line}')
        content.insert(insert_line, new_str)
        new_content = '\n'.join(content)
        p.write_text(new_content)
        return f'Inserted text at line {insert_line} in {p}'
    except: return explain_exc('inserting text')

In [ ]:
insert(f, 0, 'Let\'s add a new line')
print(view(f, nums=True))

     1 │ Let's add a new line
     2 │ Hello, world!


In [ ]:
#| export
@llmtool
def str_replace(
    path: str, # Path to the file to modify
    old_str: str, # Text to find and replace
    new_str: str # Text to replace with
) -> str:
    'Replace first occurrence of old_str with new_str in file'
    try:
        p = valid_path(path)
        content = p.read_text()
        count = content.count(old_str)
        if count == 0: return f'Error: Text "{truncstr(old_str, 10)}" not found in file'
        if count > 1: return f'Error: Multiple matches found ({count}) of "{truncstr(old_str, 10)}"'
        new_content = content.replace(old_str, new_str, 1)
        p.write_text(new_content)
        return f'Replaced text in {p}'
    except: return explain_exc('replacing text')

In [ ]:
print(str_replace(f, 'new line','new line:'))
print(view(f, nums=True))

Replaced text in /Users/jhoward/aai-ws/fastcore/nbs/test.txt
     1 │ Let's add a new line:
     2 │ Hello, world!


In [ ]:
str_replace(f, 'missing line', '')

'Error: Text "missing l…" not found in file'

In [ ]:
#| export
@llmtool
def strs_replace(
    path:str, # Path to the file to modify
    old_strs:list[str], # List of strings to find and replace
    new_strs:list[str], # List of replacement strings (must match length of old_strs)
):
    "Replace for each str pair in old_strs,new_strs"
    res = [str_replace(path, old, new) for (old,new) in zip(old_strs,new_strs)]
    return 'Results for each replacement:\n' + '; '.join(res)

In [ ]:
print(strs_replace(f, ["add a new line", "world!"], ["just say", "friends!\nNice to see you."]))
print(view(f, nums=True))

Results for each replacement:
Replaced text in /Users/jhoward/aai-ws/fastcore/nbs/test.txt; Replaced text in /Users/jhoward/aai-ws/fastcore/nbs/test.txt
     1 │ Let's just say:
     2 │ Hello, friends!
     3 │ Nice to see you.


In [ ]:
print(strs_replace(f, ["a missing", "and shouldn't polute the context"], ["", ""]))
print(view(f, nums=True))

Results for each replacement:
Error: Text "a missing" not found in file; Error: Text "and shoul…" not found in file
     1 │ Let's just say:
     2 │ Hello, friends!
     3 │ Nice to see you.


In [ ]:
#| export
@llmtool
def replace_lines(
    path:str, # Path to the file to modify
    start_line:int, # Starting line number to replace (1-based indexing)
    end_line:int, # Ending line number to replace (1-based indexing, inclusive)
    new_content:str, # New content to replace the specified lines
):
    "Replace lines in file using start and end line-numbers (index starting at 1)"
    try:
        p = valid_path(path)
        content = p.readlines()
        if not new_content.endswith('\n'): new_content+='\n'
        content[start_line-1:end_line] = [new_content]
        p.write_text(''.join(content))
        return f"Replaced lines {start_line} to {end_line}."
    except: return explain_exc('replacing lines')

In [ ]:
replace_lines('test.txt', 1, 2, 'Replaced first two lines')
print(view('test.txt', nums=True))

     1 │ Replaced first two lines
     2 │ Nice to see you.


In [ ]:
replace_lines('missing.txt', 1, 2, 'Replaced first two lines').replace(os.getcwd(), '/path')

'Error: File not found: /path/missing.txt'

In [ ]:
#| export
@llmtool
def move_lines(
    path: str,  # Path to the file to modify
    start_line: int,  # Starting line number to move (1-based)
    end_line: int,  # Ending line number to move (1-based, inclusive)
    dest_line: int,  # Destination line number (1-based, where lines will be inserted before)
) -> str:
    "Move lines from start_line:end_line to before dest_line"
    try:
        p = valid_path(path)
        lines = p.read_text().splitlines()
        ensure(1 <= start_line <= end_line <= len(lines), f"Invalid range {start_line}-{end_line}")
        ensure(1 <= dest_line <= len(lines) + 1, f"Invalid destination {dest_line}")
        ensure(not(start_line <= dest_line <= end_line + 1), "Destination within source range")
        
        chunk = lines[start_line-1:end_line]
        del lines[start_line-1:end_line]
        # Adjust dest if it was after the removed chunk
        if dest_line > end_line: dest_line -= len(chunk)
        lines[dest_line-1:dest_line-1] = chunk
        p.write_text('\n'.join(lines) + '\n')
        return f"Moved lines {start_line}-{end_line} to line {dest_line}"
    except: return explain_exc()

The `move_lines` function relocates a range of lines within a file to a new position. It handles the tricky index adjustment when the destination is after the removed chunk.

Let's test it by creating a simple 5-line file:

In [ ]:
create('move_test.txt', 'Line 1\nLine 2\nLine 3\nLine 4\nLine 5', overwrite=True)
print(view('move_test.txt', nums=True))

     1 │ Line 1
     2 │ Line 2
     3 │ Line 3
     4 │ Line 4
     5 │ Line 5


Move lines 4-5 up to before line 2:

In [ ]:
print(move_lines('move_test.txt', 4, 5, 2))
print(view('move_test.txt', nums=True))

Moved lines 4-5 to line 2
     1 │ Line 1
     2 │ Line 4
     3 │ Line 5
     4 │ Line 2
     5 │ Line 3


Move lines down — moving lines 1-2 to the end (line 6) correctly adjusts the destination index after removal:

In [ ]:
print(move_lines('move_test.txt', 1, 2, 6))
print(view('move_test.txt', nums=True))

Moved lines 1-2 to line 4
     1 │ Line 5
     2 │ Line 2
     3 │ Line 3
     4 │ Line 1
     5 │ Line 4


Error handling — destination within source range, invalid line ranges, and invalid destinations are all caught:

In [ ]:
print(move_lines('move_test.txt', 2, 3, 3))  # dest within source range
print(move_lines('move_test.txt', 10, 12, 1))  # invalid range
print(move_lines('move_test.txt', 1, 2, 99))  # invalid destination
print(move_lines('mising.txt', 1, 2, 99).replace(os.getcwd(), '/path'))  # missing file

Error: Destination within source range
Error: Invalid range 10-12
Error: Invalid destination 99
Error: File not found: /path/mising.txt


In [ ]:
Path('move_test.txt').unlink()

In [ ]:
f.unlink()

In [ ]:
#| export
def get_callable():
    "Return callable objects defined in caller's module"
    import inspect
    g = inspect.currentframe().f_back.f_globals
    return {
        f:o for f,o in g.items()
        if callable(o) and hasattr(o, '__module__') and o.__module__ == '__main__' and not f.startswith('_')
    }

## Export -

In [ ]:
#| hide
#| eval: false
from nbdev import nbdev_export
nbdev_export()